## Decorators 101
A decorator is a callable that takes another function as argument (the decorated function).

A decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In other words:
```python
@decorate
def target():
    print('abc')
```
 is equivalent to 
 ```python
 def target():
    print('abc')
target = decorate(target)
```

In [1]:
def deco(func):
    def inner():
        print('inner')
    return inner

@deco
def target():
    print('target')

target()

inner


## When Python Executes Decorators
A key feature of decorators is that they run **right after** the decorated function is defined.

In [2]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1')

def f2():
    print('running f2')

# decorator has been runned before
print('registry ->', registry)
f1()
f2()

running register(<function f1 at 0x7f2b97fcb1f0>)
registry -> [<function f1 at 0x7f2b97fcb1f0>]
running f1
running f2


## Variable Scope Rules

In [3]:
b = 6 
def f1(a):
    print(a)
    print(b)

f1(3)

3
6


but surprisingly

In [4]:
b = 6
# Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local.
def f2(a):
    print(a)
    print(b) # Variable `b` is local, because it is assigned a value in the body of the function
    b = 9

try:    
    f2(3)
except UnboundLocalError as e:
    print(e)

3
local variable 'b' referenced before assignment


In [5]:
b = 6
# Use the global declaration
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)
b # b is also changed

3
6


9

## Closures
A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there.

In [6]:
def make_averager():
    # closure
    series = []

    def averager(new_value):
        # Within `averager`, `series` is a free variable, meaning a variable that is not bound in the local scope.
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [7]:
avg.__code__.co_varnames, avg.__code__.co_freevars

(('new_value', 'total'), ('series',))

-  The value for `series` is kept in the `__closure__` attribute of the returned function `avg` 
- Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co_freevars`.
- These items are cells, and they have an attribute called `cell_contents` where the actual value can be found.

In [8]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## The nonlocal Declaration
A better implementation for `avg`

In [9]:
def make_averager():
    # only store the total and the number of items
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total # just like we need to declare global variations in regular functions
        count += 1
        total += new_value
        return total / count

    return averager
    
avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

## Implementing a Simple Decorator


In [10]:
import time

def clock(func):
    def clocked(*args):
        # record start time
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ','.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

@clock
def snooze(seconds):
    '''sleep function'''
    time.sleep(seconds)

snooze(0.123)

[0.12433838s] snooze(0.123) -> None


But we found that our decorator does not support keyword arguments, and it masks the `__name__` and `__doc__` of the decorated function.

In [11]:
try:
    snooze(seconds=0.123)
except TypeError as e:
    print(e)
snooze.__name__, snooze.__doc__

clocked() got an unexpected keyword argument 'seconds'


('clocked', None)

We can use `functools.wraps` decorator to copy the relevant attributes from func to clocked

In [12]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        # record start time
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        arg_lst.extend(repr(arg) for arg in args)
        arg_lst.extend(f'{k} = {v!r}' for k, v in kwargs.items())
        arg_str = '.'.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

@clock
def snooze(seconds):
    '''sleep function'''
    time.sleep(seconds)

snooze(0.123)
snooze(seconds=0.123) # keyword version
snooze.__name__, snooze.__doc__ 

[0.12351428s] snooze(0.123) -> None
[0.12315554s] snooze(seconds = 0.123) -> None


('snooze', 'sleep function')

## Decorators in the standard library
Some of the most interesting decorators in the standard library are `cache`, `lru_cache`, and `singledispatch`—all from the functools module. 
### Memoization with `functools.cache`

In [13]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

dummy = fibonacci(6) # The waste is obvious

[0.00000088s] fibonacci(0) -> 0
[0.00000127s] fibonacci(1) -> 1
[0.00167361s] fibonacci(2) -> 1
[0.00000084s] fibonacci(1) -> 1
[0.00000144s] fibonacci(0) -> 0
[0.00000138s] fibonacci(1) -> 1
[0.00046303s] fibonacci(2) -> 1
[0.00090680s] fibonacci(3) -> 2
[0.00303409s] fibonacci(4) -> 3
[0.00000054s] fibonacci(1) -> 1
[0.00000072s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00034883s] fibonacci(2) -> 1
[0.00184406s] fibonacci(3) -> 2
[0.00000070s] fibonacci(0) -> 0
[0.00000109s] fibonacci(1) -> 1
[0.00045326s] fibonacci(2) -> 1
[0.00000071s] fibonacci(1) -> 1
[0.00000101s] fibonacci(0) -> 0
[0.00000147s] fibonacci(1) -> 1
[0.00043502s] fibonacci(2) -> 1
[0.00086659s] fibonacci(3) -> 2
[0.00176410s] fibonacci(4) -> 3
[0.00412015s] fibonacci(5) -> 5
[0.00757122s] fibonacci(6) -> 8


In [14]:
# Avoid wastes
import functools

@functools.cache # This line works with Python 3.9 or later.
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

dummy = fibonacci(6)

[0.00000080s] fibonacci(0) -> 0
[0.00000139s] fibonacci(1) -> 1
[0.00087894s] fibonacci(2) -> 1
[0.00000238s] fibonacci(3) -> 2
[0.00138011s] fibonacci(4) -> 3
[0.00000176s] fibonacci(5) -> 5
[0.00187122s] fibonacci(6) -> 8


### Using `lru_cache`

The `functools.cache` decorator is actually a simple wrapper around the older `functools.lru_cache` function, which is more flexible and compatible with Python 3.8 and earlier versions.

The main advantage of `@lru_cache` is that its memory usage is bounded by the `maxsize` parameter, which has a rather conservative default value of 128—which means the cache will hold at most 128 entries at any time, meaning that older entries that have not been read for a while are discarded to make room for new ones. like a queue

@lru_cache # only avaliable since python 3.8

@lru_cache() # available since Python 3.2

## Parametrized Decorator


In [15]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({arg_str}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*args, **kwargs):
            t0 = time.perf_counter()
            _result = func(*args, **kwargs)
            elapsed = time.perf_counter() - t0
            arg_lst = []
            arg_lst.extend(repr(arg) for arg in args)   
            arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
            arg_str = ','.join(arg_lst)
            name = func.__name__
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()  
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(seconds=.123)

[0.12360874s] snooze(seconds=0.123) -> None
[0.12312723s] snooze(seconds=0.123) -> None
[0.12315331s] snooze(seconds=0.123) -> None


In [16]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12316067799838493s
snooze: 0.12315280300390441s
snooze: 0.12314689999766415s


### A class-based clock decorator

In [17]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({arg_str}) -> {result}'

class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt
    
    def __call__(self, func):
        def clocked(*args, **kwargs):
            t0 = time.perf_counter()
            _result = func(*args, **kwargs)
            elapsed = time.perf_counter() - t0
            arg_lst = []
            arg_lst.extend(repr(arg) for arg in args)   
            arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
            arg_str = ','.join(arg_lst)
            name = func.__name__
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12316322100377874s
snooze: 0.12315250300162006s
snooze: 0.1231515089966706s
